In [11]:
# import tensorflow as tf
# from tensorflow.keras.optimizers import RMSprop
# from tensorflow.keras.utils import to_categorical

In [12]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping


In [15]:
# Reading the given dataset
import pandas as pd
spam = pd.read_csv("https://raw.githubusercontent.com/arjun0200/Datasets/main/spam.csv", encoding = 'latin-1', skipinitialspace = True, usecols = ['v1','v2'])
spam.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [16]:
X = spam.v2
Y = spam.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
print(Y)
Y = Y.reshape(-1,1)
print(type(X))

[0 0 1 ... 0 0 0]
<class 'pandas.core.series.Series'>


In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20)

In [18]:
X_train.shape

(4457,)

In [22]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
print(sequences)
sequences_matrix = sequence.pad_sequences(sequences, maxlen = max_len)
sequences_matrix

[[31, 3, 196, 22, 48, 1, 29, 140], [306, 50, 460, 20, 108, 295, 22, 3, 71, 21, 22, 3, 167, 27, 3, 100, 15, 10, 1, 420, 11, 7, 67, 3, 4, 461, 49, 5, 719], [5, 332, 4, 356, 24, 319, 15, 5, 15, 440, 263, 26, 319, 15, 5, 15, 57, 263], [151, 814, 637, 84], [421, 5, 44, 43, 284, 571, 33, 4, 257, 7, 220, 30, 240, 2, 11, 174, 44, 43, 83, 241, 12, 20, 333], [2, 169, 145, 49, 720, 2, 870, 441, 608, 73, 7, 67, 2, 334, 178, 65, 251, 386, 89, 42, 679], [142, 13, 103, 721, 25, 130, 6, 80, 2, 462, 2, 5, 307, 546, 815, 33, 335, 12, 48, 16, 5, 103, 462, 242, 48, 18], [50, 131, 80, 2, 422], [74, 221, 13, 816, 127, 112, 609, 74, 13, 12, 55, 871, 423, 2, 13, 264], [136, 58, 80, 81, 2], [26, 163, 325, 84, 136, 56, 6, 515, 480, 136], [39, 77, 279, 34, 119, 361], [463, 131, 28, 75, 96, 361], [320, 1, 275, 23, 547, 146, 516, 84], [50, 22, 3, 235, 3, 4, 124, 872], [71, 8, 39, 9, 11, 7], [342, 59, 336, 722, 17, 2, 67, 9, 11, 2, 35, 14, 127, 4, 326], [946, 87, 37, 2, 333, 20, 3, 95, 321, 817, 10, 45, 572, 28, 32

array([[  0,   0,   0, ...,   1,  29, 140],
       [  0,   0,   0, ...,  49,   5, 719],
       [  0,   0,   0, ...,  15,  57, 263],
       ...,
       [  0,   0,   0, ..., 681, 700,   3],
       [  0,   0,   0, ...,   2,  34, 917],
       [  0,   0,   0, ..., 571,   7,  62]], dtype=int32)

In [25]:
def RNN():
  inputs = Input(name = 'inputs', shape = [max_len])
  layer = Embedding(max_words, 50, input_length = max_len)(inputs)
  layer = LSTM(64)(layer)  # LongShortTermMemory
  layer = Activation('relu')(layer)
  layer = Dropout(0.5)(layer)  #Improvision of model using Dropouts
  layer = Dense(1, name = 'out_layer')(layer)  #final fully connected dense layer
  layer = Activation('sigmoid')(layer)  # output layer
  model = Model(inputs = inputs, outputs = layer)
  return model

In [26]:
model = RNN()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 activation (Activation)     (None, 64)                0         
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 out_layer (Dense)           (None, 1)                 65        
                                                                 
 activation_1 (Activation)   (None, 1)                 0     

In [27]:
model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(), metrics = ['accuracy'])

In [30]:
model.fit(sequences_matrix, Y_train, batch_size = 128, epochs = 10, validation_split = 0.2, callbacks = [EarlyStopping(monitor = 'val_loss', min_delta = 0.0001)])
#EarlyStopping when similar loss occur it stop

Epoch 1/10
28/28 [==============================] - 6s 207ms/step - loss: 0.1049 - accuracy: 0.9781 - val_loss: 0.0823 - val_accuracy: 0.9787
Epoch 2/10
28/28 [==============================] - 6s 204ms/step - loss: 0.0627 - accuracy: 0.9860 - val_loss: 0.0701 - val_accuracy: 0.9765


In [44]:
# max_words = 1000
# max_len = 150
# tok = Tokenizer(num_words = max_words)
# tok.fit_on_texts(X_test)
# sequences = tok.texts_to_sequences(X_test)
# print(sequences)
# sequences_matrix = sequence.pad_sequences(sequences, maxlen = max_len)
# sequences_matrix

In [45]:
#model.evaluate(sequences_matrix, Y_test)

-0.039587246413012256